In [1]:
from pandas import read_csv
import numpy as np
from collections import Counter
from pprint import pprint
import math
import csv

In [2]:
dataFrame = read_csv('data/training.csv',header = None)
#print(dataFrame)

MemoryError: 

In [ ]:
trainingLabels = dataFrame.iloc[:,-1].values
totalDocuments = len(trainingLabels)
print(trainingLabels)
print("the length of trainningLabels is ====  " + str(totalDocuments))

In [ ]:
dataFrame.drop(dataFrame.columns[[-1,0]],axis = 1, inplace=True)
#print(dataFrame)

In [ ]:
trainingClassCount = Counter()
for val in trainingLabels:
    trainingClassCount[val] += 1
   
pprint(trainingClassCount)
print(len(trainingClassCount))

In [ ]:
allLabels = np.unique(trainingLabels)
print(allLabels)
totalClassCount = len(allLabels)
print(totalClassCount)

In [ ]:
sortedClassCount = sorted(trainingClassCount.items())
#print(sortedClassCount)

classCount = [y for x,y in sortedClassCount]
#print(classCount)
logPriorProb = [math.log(x/totalDocuments) for x in classCount]
#pprint(logPriorProb)

In [ ]:
trainingMatrix = np.array(dataFrame.values)
totalWords = len(trainingMatrix[0])
#print(totalWords)
#print(trainingMatrix)
#print("the size of training matrix is:    "  + str(len(trainingMatrix)) + "  X  " +  str(len(trainingMatrix[0])))

In [ ]:
dividedOnClass = [[x for x, y in zip(trainingMatrix,trainingLabels) if y == c] for c in allLabels]


#pprint(dividedOnClass)
#print(len(dividedOnClass))


In [ ]:
tempTable = [np.array(val).sum(axis=0)for val in dividedOnClass]

#pprint (tempTable)

In [ ]:
smoother = 1 / totalWords
#print(smoother)
smoothCount = np.array(tempTable) + smoother
#pprint(smoothCount)
#print(len(smoothCount))

In [ ]:

    
totalWordInEachClass = [sum(val) for val in tempTable]
#pprint(totalWordInEachClass)

In [ ]:
totalWordInEachClassSmooth = np.array(totalWordInEachClass) + 1
#print(totalWordInEachClassSmooth)


In [ ]:
likelihood = np.array([ x / y for x , y in zip (smoothCount,totalWordInEachClassSmooth)])
#pprint(likelihood)
#print(len(likelihood))

In [ ]:
logLikelihood = np.array([np.log(val) for val in likelihood])
#pprint(logLikelihood)
#print(len(logLikelihood))

In [ ]:
#pprint(logPriorProb)
#pprint(logLikelihood)

In [ ]:
testingDataFrame = read_csv('data/testing.csv', header = None)
testingId = testingDataFrame.iloc[:,0].values
#pprint(testingDataFrame)
#print(testingId)
testingDataFrame.drop(testingDataFrame.columns[0,],axis=1,inplace=True)


In [ ]:


#print(testingDataFrame)


In [ ]:
testingVals = np.array(testingDataFrame.values)
#print(testingVals)
probEstimation = [logLikelihood.dot(val) for val in testingVals]
#print(probEstimation)




In [ ]:
totalProbEstimation = [[x + y for x,y in zip(logPriorProb, val)] for val in probEstimation]
#print(totalProbEstimation)

In [ ]:
prediction = [val.index(max(val))+1 for val in totalProbEstimation]
#print(prediction)

In [ ]:
csvData = [ [docId , p]  for docId,p in zip (testingId,prediction)]
pprint(csvData)


In [ ]:
csvData = [['id','class']] + csvData
with open('data/submission.csv', 'w',newline='') as csvFile:
    
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()